# Home Credit Dataset Feature Engineering 

Part of my University Course "Advanced Business Analytics with R" at the University of Texas at Dallas - 
Naveen Jindal School of Management, I am required to work and present a project. For the same reason, I chose the Home Credit Deafult risk dataset which was a Kaggle Competition sometime back. This dataset came with a whole lot of complexity, in terms of the feature engineering. The main aim of this competition was to predict whether a person is capable of repaying a loan or not. 

The dataset consists of data relating to the applicants, their previous applications, credit bureau data and so on. The feature engineering in this case was tough because for each applicant can have mutiple previous loan applications, plus mutiple credit bureau reports and hence, each individual applicant would have multiple rows for the current application when combining all the datasets. There is a need to have the additional data apart from the applicant data as that would be a huge deciding factor in the payment capabilities.     

The feature engineering is a very time consuming task in this case and hence Dask as well as Featuretools came in handy to speed up the process and obtain the final feature set as desired.

In [47]:
import pandas as pd 
import numpy as np 
import os as os 
import sys
import psutil

distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:52239 remote=tcp://127.0.0.1:56144>


Loading the datasets

In [49]:
application = pd.read_csv("Application.csv")
bureau = pd.read_csv("bureau.csv").replace({365243: np.nan})
bureau_balance = pd.read_csv("bureau_balance.csv").replace({365243: np.nan})
credit_card_balance = pd.read_csv("credit_card_balance.csv").replace({365243: np.nan})
installment_payments = pd.read_csv("installments_payments.csv").replace({365243: np.nan})
POS_CASH_balance = pd.read_csv("POS_CASH_balance.csv").replace({365243: np.nan})
previous_application = pd.read_csv("previous_application.csv").replace({365243: np.nan})

In [10]:
bureau_balance = bureau_balance.merge(bureau[['SK_ID_CURR', 'SK_ID_BUREAU']], 
                                      on = 'SK_ID_BUREAU', how = 'left')

Converting data types to reduce memory usage

In [12]:
print(f"""Total memory before converting types: \
{round(np.sum([x.memory_usage().sum() / 1e9 for x in 
[application, bureau, bureau_balance, credit_card_balance, POS_CASH_balance, previous_application, installment_payments ]]), 2)} gb.""")

Total memory before converting types: 4.03 gb.


In [51]:
def convert_types(df):
    """Convert pandas data types for memory reduction."""
    
    # Iterate through each column
    for c in df:
        
        # Convert ids and booleans to integers
        if ('SK_ID' in c):
            df[c] = df[c].fillna(0).astype(np.int32)
            
        # Convert objects to category
        elif (df[c].dtype == 'object') and (df[c].nunique() < df.shape[0]):
            df[c] = df[c].astype('category')
        
        # Booleans mapped to integers
        elif set(df[c].unique()) == {0, 1}:
            df[c] = df[c].astype(bool)
        
        # Float64 to float32
        elif df[c].dtype == float:
            df[c] = df[c].astype(np.float32)
            
        # Int64 to int32
        elif df[c].dtype == int:
            df[c] = df[c].astype(np.int32)
        
    return df

In [52]:
application = convert_types(application)
bureau = convert_types(bureau)
bureau_balance = convert_types(bureau_balance)
credit_card_balance = convert_types(credit_card_balance)
installment_payments = convert_types(installment_payments)
POS_CASH_balance = convert_types(POS_CASH_balance)
previous_application = convert_types(previous_application)

In [53]:
print(f"""Total memory after converting types: \
{round(np.sum([x.memory_usage().sum() / 1e9 for x in 
[application, bureau, bureau_balance, credit_card_balance, POS_CASH_balance, previous_application, installment_payments ]]), 2)} gb.""")

Total memory after converting types: 1.85 gb.


Creating partitions to run the Automated Feature Engineering quicker 

In [ ]:
def create_partition(client_list, partition_number):
    """Creates and saves a dataset with users in user_list"""
    directory = os.path.join(os.getenv('programdata'), '/d%d' % (partition_number + 1))
    if not os.path.isdir(directory):
        os.makedirs(directory)

    
        application_subset = application[application.index.isin(client_list)].copy().reset_index()
        bureau_subset = bureau[bureau.index.isin(client_list)].copy().reset_index()
        bureau_balance_subset = bureau_balance[bureau_balance.index.isin(client_list)].copy().reset_index(drop = True)
        POS_CASH_balance_subset = POS_CASH_balance[POS_CASH_balance.index.isin(client_list)].copy().reset_index(drop = True)
        credit_card_balance_subset = credit_card_balance[credit_card_balance.index.isin(client_list)].copy().reset_index(drop = True)
        previous_application_subset = previous_application[previous_application.index.isin(client_list)].copy().reset_index()
        installment_payments_subset = installment_payments[installment_payments.index.isin(client_list)].copy().reset_index(drop = True)
    
    # Save subset data with partition number as identifier
        application_subset.to_csv('%s/application.csv' % directory, index = False)
        bureau_subset.to_csv('%s/bureau.csv' % directory, index = False)
        bureau_balance_subset.to_csv('%s/bureau_balance.csv' % directory, index = False)
        POS_CASH_balance_subset.to_csv('%s/POS_CASH_balance.csv' % directory, index = False)
        credit_card_balance_subset.to_csv('%s/credit_card_balance.csv' % directory, index = False)
        previous_application_subset.to_csv('%s/previous_application.csv' % directory, index = False)
        installment_payments_subset.to_csv('%s/installment_payments.csv' % directory, index = False)



In [ ]:
# Chunk size for 104 sections
chunk_size = len(application) // 103

# Create list of lists of clients in each section
client_lists = [application[i:i + chunk_size].index for i in range(0, len(application), chunk_size)]

In [21]:
from itertools import chain

# Sanity check that we have not missed any ids
print('Number of ids in id_list:         {}.'.format(len(list(chain(*client_lists)))))
print('Total length of application data: {}.'.format(len(application)))        

Number of ids in id_list:         307507.
Total length of application data: 307507.


In [ ]:
for i, client_list in enumerate(client_lists):
    create_partition(client_list, i)    

I have used Deep Feature Synthesis, so that, I can get all the features based on the main application data and do it quickly, 
because doing through pandas group by and other arithmetic functions is very time consuming. The deep feature synthesis through 
the featuretools package automatically maps the feature and performs the required function a lot more efficiently. 

In [22]:
import featuretools as ft
import featuretools.variable_types as vtypes

I have mapped the entity relations for the data through the indexed value (SK_ID_CURR).

In [23]:
def entityset(path):
    
    partition_num = path[4:]
     
    application = pd.read_csv("%s/application.csv" % path)
    bureau = pd.read_csv("%s/bureau.csv" % path)
    bureau_balance = pd.read_csv("%s/bureau_balance.csv" % path)
    credit_card_balance = pd.read_csv("%s/credit_card_balance.csv" % path)
    installment_payments = pd.read_csv("%s/installment_payments.csv" % path)
    POS_CASH_balance = pd.read_csv("%s/POS_CASH_balance.csv" % path)
    previous_application = pd.read_csv("%s/previous_application.csv" % path)
    
    #creating entityset
    es = ft.EntitySet(id = "credit")
    
    #creating entity from application
    es = es.entity_from_dataframe(entity_id="application", dataframe = application, index = "SK_ID_CURR")
    
    #creating entity from previous
    es = es.entity_from_dataframe(entity_id="previous_application", dataframe = previous_application, index = "SK_ID_PREV")
    
    #creating entity from bureau
    es = es.entity_from_dataframe(entity_id = 'bureau', dataframe = bureau, index = 'SK_ID_BUREAU')
    
    #creating entity from bureau_balance with no unique index
    es = es.entity_from_dataframe(entity_id = 'bureau_balance', dataframe = bureau_balance, 
                            make_index = True, index = 'bureaubalance_id10')
    
    #creating entity from credit_card_balance with no unique index
    es = es.entity_from_dataframe(entity_id = 'credit_card_balance', dataframe = credit_card_balance, 
                            make_index = True, index = 'credit_card_id10')
    
    #creating entity from installment with no unique index
    es = es.entity_from_dataframe(entity_id = 'installment_payments', dataframe = installment_payments, 
                            make_index = True, index = 'installment_payments_id10')
    
    #creating entity from POS_CASH_balance with no unique index
    es = es.entity_from_dataframe(entity_id = 'POS_CASH_balance', dataframe = POS_CASH_balance, 
                            make_index = True, index = 'POS_CASH_balance_id10')
    
    #relationship between application and bureau
    app_r = ft.Relationship(es["application"]["SK_ID_CURR"], 
                       es["bureau"]["SK_ID_CURR"])
    
    # Relationship between bureau and bureau balance
    bur_r = ft.Relationship(es['bureau']['SK_ID_BUREAU'], es['bureau_balance']['SK_ID_BUREAU'])
    
    # Relationship between current app and previous apps
    prev_r = ft.Relationship(es['application']['SK_ID_CURR'], es['previous_application']['SK_ID_CURR'])
    
    # Relationships between previous apps and cash, installments, and credit
    prev_cash_r = ft.Relationship(es['previous_application']['SK_ID_PREV'], es['POS_CASH_balance']['SK_ID_PREV'])
    prev_inst_r = ft.Relationship(es['previous_application']['SK_ID_PREV'], es['installment_payments']['SK_ID_PREV'])
    prev_cred_r = ft.Relationship(es['previous_application']['SK_ID_PREV'], es['credit_card_balance']['SK_ID_PREV'])
    
    es = es.add_relationships([app_r, bur_r, prev_r, prev_cash_r, prev_inst_r, prev_cred_r])
    
    return ({'es': es, 'num': partition_num})
    
    
    

I have tested the Deep Feature Synthesis for one parition so that it can then be run for every other partition.

In [24]:
es1_dict = entityset('C:/d1')
es1_dict['es']

Entityset: credit
  Entities:
    application [Rows: 2985, Columns: 87]
    previous_application [Rows: 14044, Columns: 37]
    bureau [Rows: 13899, Columns: 17]
    bureau_balance [Rows: 143218, Columns: 5]
    credit_card_balance [Rows: 31625, Columns: 24]
    installment_payments [Rows: 112125, Columns: 9]
    POS_CASH_balance [Rows: 83320, Columns: 9]
  Relationships:
    bureau.SK_ID_CURR -> application.SK_ID_CURR
    bureau_balance.SK_ID_BUREAU -> bureau.SK_ID_BUREAU
    previous_application.SK_ID_CURR -> application.SK_ID_CURR
    POS_CASH_balance.SK_ID_PREV -> previous_application.SK_ID_PREV
    installment_payments.SK_ID_PREV -> previous_application.SK_ID_PREV
    credit_card_balance.SK_ID_PREV -> previous_application.SK_ID_PREV

Checking the number of features built 

In [25]:
# Default primitives from featuretools
default_agg_primitives =  ['sum', 'count', 'min', 'max', 'mean', 'mode']

# DFS with specified primitives
feature_names = ft.dfs(entityset = es1_dict['es'], target_entity = 'application',
                       agg_primitives=default_agg_primitives, 
                       max_depth = 2, features_only=True)

print('%d Total Features' % len(feature_names))

1180 Total Features


In [26]:
feature_names1 = ft.save_features(feature_names)

In [27]:
feature_defs = ft.load_features(feature_names1)

Function to create Feature Matrix

In [28]:
def feature_matrix_from_entityset(es_dict, feature_defs, return_fm = False):
    """Run deep feature synthesis from an entityset and feature definitions. 
    Saves feature matrix based on partition.""" 
    os.getenv('programdata')
    
    # Extract the entityset
    es = es_dict['es']
    
    # Calculate the feature matrix and save
    feature_matrix = ft.calculate_feature_matrix(feature_defs,
                                                 entityset=es, 
                                                 n_jobs = 1, 
                                                 verbose = 0,
                                                 chunk_size = es['application'].df.shape[0])
    
    feature_matrix.to_csv('C:/Users/prith/OneDrive/Desktop/Advanced BA with R/p%s_fm.csv' % es_dict['num'], index = True,)
    
    if return_fm:
        return feature_matrix

Test with one parition to check whether the features were built

In [49]:
feature_matrix_spec, feature_names_spec = ft.dfs(entityset = es1_dict['es'], target_entity = 'application',  
                                                 agg_primitives = ['sum', 'count', 'min', 'max', 'mean', 'mode'], 
                                                 max_depth = 2, features_only = False, verbose = True)

Built 1180 features
Elapsed: 00:04 | Progress:  28%|████████████████████████▏                                                              

distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:54921 remote=tcp://127.0.0.1:54841>


Elapsed: 00:47 | Progress: 100%|███████████████████████████████████████████████████████████████████████████████████████


In [50]:
feature_matrix_spec.head()

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,MODE(credit_card_balance.previous_application.NAME_PRODUCT_TYPE),MODE(credit_card_balance.previous_application.FLAG_LAST_APPL_PER_CONTRACT),MODE(credit_card_balance.previous_application.NAME_PORTFOLIO),MODE(credit_card_balance.previous_application.NAME_SELLER_INDUSTRY),MODE(credit_card_balance.previous_application.WEEKDAY_APPR_PROCESS_START),MODE(credit_card_balance.previous_application.NAME_CONTRACT_STATUS),MODE(credit_card_balance.previous_application.PRODUCT_COMBINATION),MODE(credit_card_balance.previous_application.NAME_CASH_LOAN_PURPOSE),MODE(credit_card_balance.previous_application.NAME_CONTRACT_TYPE),MODE(credit_card_balance.previous_application.CHANNEL_TYPE)
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,True,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100003,False,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100004,False,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100006,False,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,x-sell,Y,Cards,XNA,THURSDAY,Approved,Card X-Sell,XAP,Revolving loans,Credit and cash offices
100007,False,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
feature_matrix_spec.shape

(2985, 1180)

Clearing up System memory to run on Dask

In [29]:
import gc

# Free up all system memory
gc.enable()
del application, bureau, bureau_balance, previous_application, credit_card_balance, POS_CASH_balance, installment_payments
gc.collect()

0

I have used Dask to run the full Deep feature Synthesis as it parallelizes the total feature calculation process 
and it can be run quickly. It took me a total of 3 hours to get the final feature matrix. Dask utilizes all the cores from the system and runs the entire operation in parallel, thus acheiving the process quicker without the external use of a supercomputer. 

In [30]:
import dask.bag as db
from dask.distributed import Client

# Use all 8 cores
client = Client(processes = True)

In [31]:
client.ncores()

{'tcp://127.0.0.1:56164': 2,
 'tcp://127.0.0.1:56168': 2,
 'tcp://127.0.0.1:56170': 2,
 'tcp://127.0.0.1:56172': 2}

Path Creation to run the process

In [33]:
paths = ['C:/d%d' %  i for i in range(1, 105)]
paths

['C:/d1',
 'C:/d2',
 'C:/d3',
 'C:/d4',
 'C:/d5',
 'C:/d6',
 'C:/d7',
 'C:/d8',
 'C:/d9',
 'C:/d10',
 'C:/d11',
 'C:/d12',
 'C:/d13',
 'C:/d14',
 'C:/d15',
 'C:/d16',
 'C:/d17',
 'C:/d18',
 'C:/d19',
 'C:/d20',
 'C:/d21',
 'C:/d22',
 'C:/d23',
 'C:/d24',
 'C:/d25',
 'C:/d26',
 'C:/d27',
 'C:/d28',
 'C:/d29',
 'C:/d30',
 'C:/d31',
 'C:/d32',
 'C:/d33',
 'C:/d34',
 'C:/d35',
 'C:/d36',
 'C:/d37',
 'C:/d38',
 'C:/d39',
 'C:/d40',
 'C:/d41',
 'C:/d42',
 'C:/d43',
 'C:/d44',
 'C:/d45',
 'C:/d46',
 'C:/d47',
 'C:/d48',
 'C:/d49',
 'C:/d50',
 'C:/d51',
 'C:/d52',
 'C:/d53',
 'C:/d54',
 'C:/d55',
 'C:/d56',
 'C:/d57',
 'C:/d58',
 'C:/d59',
 'C:/d60',
 'C:/d61',
 'C:/d62',
 'C:/d63',
 'C:/d64',
 'C:/d65',
 'C:/d66',
 'C:/d67',
 'C:/d68',
 'C:/d69',
 'C:/d70',
 'C:/d71',
 'C:/d72',
 'C:/d73',
 'C:/d74',
 'C:/d75',
 'C:/d76',
 'C:/d77',
 'C:/d78',
 'C:/d79',
 'C:/d80',
 'C:/d81',
 'C:/d82',
 'C:/d83',
 'C:/d84',
 'C:/d85',
 'C:/d86',
 'C:/d87',
 'C:/d88',
 'C:/d89',
 'C:/d90',
 'C:/d91',
 'C:/d92

The bag operation is used to parallelize the tasks

In [34]:
# Create a bag object
b = db.from_sequence(paths)

# Map entityset function
b = b.map(entityset)

# Map feature matrix function
b = b.map(feature_matrix_from_entityset, feature_defs = feature_defs)
    
b

dask.bag<feature..., npartitions=104>

The final computation is performed for each partition (104 in this case) and the feature matrix is obtained with all the features.

In [35]:
b.compute()

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

I have then combined all the partitioned datasets to return the final feature matrix conatining the total applicant data along with all the features. 

In [40]:
# Base directory for feature matrices
base = 'C:/Users/prith/OneDrive/Desktop/Advanced BA with R/'
fm_paths = [base + p for p in os.listdir(base) if 'fm.csv' in p]

In [41]:
fms = [pd.read_csv(path) for path in fm_paths]

In [42]:
feature_matrix = pd.concat(fms, axis = 0)

In [43]:
print("Final feature matrix Shape:", feature_matrix.shape)

Final feature matrix Shape: (307507, 1181)


In [44]:
feature_matrix.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,MODE(credit_card_balance.previous_application.NAME_SELLER_INDUSTRY),MODE(credit_card_balance.previous_application.NAME_CASH_LOAN_PURPOSE),MODE(credit_card_balance.previous_application.NAME_CONTRACT_STATUS),MODE(credit_card_balance.previous_application.NAME_PAYMENT_TYPE),MODE(credit_card_balance.previous_application.WEEKDAY_APPR_PROCESS_START),MODE(credit_card_balance.previous_application.NAME_TYPE_SUITE),MODE(credit_card_balance.previous_application.NAME_PORTFOLIO),MODE(credit_card_balance.previous_application.NAME_YIELD_GROUP),MODE(credit_card_balance.previous_application.PRODUCT_COMBINATION),MODE(credit_card_balance.previous_application.CODE_REJECT_REASON)
0,442380,False,Cash loans,F,N,Y,0,270000.0,790830.0,62613.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,442381,False,Cash loans,M,N,N,0,360000.0,152820.0,16177.5,...,XNA,XAP,Approved,XNA,TUESDAY,NaN,Cards,XNA,Card X-Sell,XAP
2,442382,False,Cash loans,F,N,N,0,306000.0,1042560.0,63913.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,442383,False,Cash loans,F,Y,Y,0,135000.0,1057500.0,28026.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,442384,True,Cash loans,M,N,Y,0,90000.0,513531.0,24835.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Finally, load the feature matrix to a CSV for further modelling and deriving the insights.

In [45]:
feature_matrix.to_csv('C:/Users/prith/OneDrive/Desktop/Advanced BA with R/final_feature.csv')

The taks was overall very time consuming, but, I was able to save a lot of time. Running the Deep Feature Synthesis for a dataset of this size and volume would take more than a day in a Supercomputer. But, utlizing all the cores and running the tasks in parallel make it possible to compute the entire feature in over 3 hours. 